In [1]:
import warnings
import math

import pandas as pd
import numpy as np

from jre_utils.datapath import (
    factor_data_paths,
    model_built_data_paths,
    model_ready_data_paths,
    get_derived_csv_path,
    get_derived_lpa_path,
    get_derived_plps_path,
)

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
asset_type = "building"
years_ahead = 2

metrics = {
    "weighted_mean": "unit_price_wmean",
    "weighted_median": "unit_price_wmedian",
    "mean": "unit_price_mean",
    "median": "unit_price_median",
    "weighted_mean_smoothed": "unit_price_wmean_smoothed",
    "weighted_median_smoothed": "unit_price_wmedian_smoothed",
    "mean_smoothed": "unit_price_mean_smoothed",
    "median_smoothed": "unit_price_median_smoothed",
}

dataset_paths = {
    "transactions": get_derived_csv_path(asset_type),
    "lpa": get_derived_lpa_path(),
    "plps": get_derived_plps_path()
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

# metric_key_unsmoothed = "weighted_median"
metric_key_unsmoothed = "median"
metric_unsmoothed = metrics[metric_key_unsmoothed]

metric_key = f"{metric_key_unsmoothed}_smoothed"
metric = metrics[metric_key]

metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric

In [3]:
dataset_key = "transactions"
core_path = dataset_paths[dataset_key]
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]


In [4]:
def years_since_crisis(year):
    year_ranges = {
        (1960, 1973): 1960,
        (1973, 1990): 1973,
        (1990, 1997): 1990,
        (1997, 2008): 1997,
        (2008, 2019): 2008,
        (2019, math.inf): 2019,
    }

    for range_start, range_end in year_ranges:
        if range_start <= year < range_end:
            return year - year_ranges[(range_start, range_end)]
        

In [5]:
df = pd.read_csv(core_path)
# df = get_most_active_municipalities(df, 1500)

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

# Years since crisis (Yn - 2008) or (Yn - 2020)
df["years_since_crisis"] = df["year"].apply(years_since_crisis)

In [6]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)

# Might go back and undo the new dwellings filling for unknown municipalities.
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)

In [7]:
# prepare metrics
df = df.sort_values(by=group_by_columns, ascending=True)
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(periods=years_ahead)
df = df[~df[metric_pct_chg].isna()]

In [8]:
columns = [
    metric_pct_chg,
    metric,
    metric_unsmoothed,
    "year",
    "years_since_crisis",
    "count",
    "total_traded_area",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "total_tax",
    "total_tax_growth",
    "new_dwellings",
    "existing_dwellings",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
    "area_code",  # id
    "area", # additional info
]

# add ratios and growths if necessary

df = df[columns]

In [9]:
df.shape

(18826, 25)

In [10]:
len(df["area_code"].unique())

1377

In [11]:
df.describe()

,unit_price_median_smoothed_pct_chg,unit_price_median_smoothed,unit_price_median,year,years_since_crisis,count,total_traded_area,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,total_tax,total_tax_growth,new_dwellings,existing_dwellings,net_migration_ratio,new_dwellings_ratio,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,area_code
count,18826.000000,1.882600e+04,1.882600e+04,18826.000000,18826.000000,18826.000000,18826.000000,1.882600e+04,1.741000e+04,1.741000e+04,17410.000000,17410.000000,17410.000000,1.879000e+04,18778.000000,10378.000000,1.037800e+04,18826.000000,10378.000000,18826.0,18826.000000,18826.000000,18826.000000,18826.000000
mean,-0.002124,8.790853e+04,8.867827e+04,2015.617125,4.430947,81.128014,19321.521035,9.387693e+04,4.245591e+04,1.424153e+08,2905.419252,0.003295,0.002599,1.444568e+07,0.003111,1075.371555,6.980830e+04,-0.002912,0.013005,1.0,0.924785,0.551259,0.998088,21603.518963
std,0.215441,1.700872e+05,1.707510e+05,3.973032,3.095357,197.717847,36648.558412,2.168423e+05,9.954537e+04,3.745715e+08,572.392933,0.043532,0.033404,4.239654e+07,0.047345,2404.116441,1.356907e+05,0.006135,0.006488,0.0,0.263745,0.497379,0.043689,13633.134424
min,-0.846925,1.380199e+03,1.000000e+02,2007.000000,0.000000,1.000000,65.000000,1.481000e+03,5.300000e+02,1.306821e+06,1989.133483,-0.529577,-0.480760,1.307820e+05,-0.677251,2.000000,5.053000e+03,-0.099818,0.000298,1.0,0.000000,0.000000,0.000000,1100.000000
25%,-0.118428,2.027423e+04,2.003300e+04,2012.000000,2.000000,10.000000,3940.000000,1.590925e+04,6.750250e+03,1.792493e+07,2576.276955,-0.010224,-0.007842,1.985892e+06,-0.014887,175.000000,1.836225e+04,-0.006435,0.008755,1.0,1.000000,0.000000,1.000000,11100.000000
50%,-0.015540,4.107204e+04,4.242424e+04,2016.000000,4.000000,26.000000,8865.000000,3.574000e+04,1.548500e+04,4.351653e+07,2798.032427,0.006283,0.003193,4.564943e+06,0.002282,399.500000,3.043900e+04,-0.003010,0.012268,1.0,1.000000,1.000000,1.000000,21202.500000
75%,0.087146,9.612718e+04,9.875937e+04,2019.000000,7.000000,76.000000,19775.000000,8.301350e+04,3.760575e+04,1.150992e+08,3099.449753,0.021106,0.013201,1.219805e+07,0.018414,948.750000,6.357650e+04,0.000347,0.016135,1.0,1.000000,1.000000,1.000000,32501.000000
max,5.327160,4.462737e+06,4.420875e+06,2022.000000,10.000000,4258.000000,625095.000000,3.794024e+06,1.906224e+06,7.965148e+09,12667.020000,1.066040,1.026488,8.672765e+08,2.199177,39143.000000,1.916062e+06,0.043446,0.120270,1.0,1.000000,1.000000,1.000000,47362.000000


In [12]:
df.isna().sum()

unit_price_median_smoothed_pct_chg       0
unit_price_median_smoothed               0
unit_price_median                        0
year                                     0
years_since_crisis                       0
count                                    0
total_traded_area                        0
population                               0
taxpayer_count                        1416
taxable_income                        1416
taxable_income_per_taxpayer           1416
taxable_income_growth                 1416
taxable_income_per_taxpayer_growth    1416
total_tax                               36
total_tax_growth                        48
new_dwellings                         8448
existing_dwellings                    8448
net_migration_ratio                      0
new_dwellings_ratio                   8448
migrations_is_available                  0
taxable_income_is_available              0
dwellings_is_available                   0
total_tax_is_available                   0
area_code  

In [13]:
# New
# df = df.fillna(0)
df.to_csv(model_built_data_paths[f"sequence_{dataset_key}_{metric_key}_{years_ahead}"], index=False)

In [14]:
# Historical
# df = df.fillna(0)
# df.to_csv(model_ready_data_paths[f"sequence_{dataset_key}_{metric_key}_{years_ahead}"], index=False)


In [15]:
df[df["area_code"] == 13101][["year", metric, metric_pct_chg]]

,year,unit_price_median_smoothed,unit_price_median_smoothed_pct_chg
1733,2007,2.204983e+06,0.227171
1734,2008,2.197454e+06,0.139858
1735,2009,2.069647e+06,-0.061378
1736,2010,1.825045e+06,-0.169473
1737,2011,1.620211e+06,-0.217156
1738,2012,1.684720e+06,-0.076888
1739,2013,1.947510e+06,0.202010
1740,2014,2.130575e+06,0.264646
1741,2015,2.257352e+06,0.159097
1742,2016,2.645833e+06,0.241840
